In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importamos las librerias que vamos a usar

In [3]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np
import pandas as pd

# Libreria para graficos
from matplotlib import pyplot
from sklearn.model_selection import train_test_split #libreria para dividir los datos para entrenamiento y prueba

# Modulo de optimizacion en scipy
from scipy import optimize

# modulo para cargar archivos en formato MATLAB
# from scipy.io import loadmat

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

Cargamos el dataset, tratamos algunas columnas, la columna class se convitio de texto a numeros

In [4]:
# La entrada es de 16(1-16) elementos sin contar con x0
input_layer_size  = 16

# 3 etiquetas, de 1 a 3 (tomar en cuenta que se asigna "0" a la etiqueta 3)
num_labels = 3

#  datos de entrenamiento almacenados en los arreglos X, y

data = pd.read_csv('/content/drive/MyDrive/SIS420 IA 1/datasets/Propios/star_classification.csv', delimiter=',')
#display(data)

data = data.drop(['obj_ID','rerun_ID'], axis = 1 ) #borramos datos de ID unicos y rerun_ID que tiene un solo valor repeidos

#convertimos la columna class a numeros
data['class'] = data['class'].map({'GALAXY':0,'QSO':1,'STAR':2})
print(data.shape[0])
print(data.info())
#display(data)



100000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 16 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   alpha        100000 non-null  float64
 1   delta        100000 non-null  float64
 2   u            100000 non-null  float64
 3   g            100000 non-null  float64
 4   r            100000 non-null  float64
 5   i            100000 non-null  float64
 6   z            100000 non-null  float64
 7   run_ID       100000 non-null  int64  
 8   cam_col      100000 non-null  int64  
 9   field_ID     100000 non-null  int64  
 10  spec_obj_ID  100000 non-null  float64
 11  class        100000 non-null  int64  
 12  redshift     100000 non-null  float64
 13  plate        100000 non-null  int64  
 14  MJD          100000 non-null  int64  
 15  fiber_ID     100000 non-null  int64  
dtypes: float64(9), int64(7)
memory usage: 12.2 MB
None


Separamos datos de entrenamiento(80%) y prueba(20%) por cada clase, ya que cada clase tiene un diferente numero de datos

In [5]:
y_tem = data["class"]

data_class_0 = data[y_tem == 0]
train_class_0, test_class_0 = train_test_split(data_class_0, test_size=0.2, random_state=42)

data_class_1 = data[y_tem == 1]
train_class_1, test_class_1 = train_test_split(data_class_1, test_size=0.2, random_state=42)

data_class_2 = data[y_tem == 2]
train_class_2, test_class_2 = train_test_split(data_class_2, test_size=0.2, random_state=42)


In [6]:
print("para la clase 0 se tiene la cantidad total de: ",data_class_0.shape[0], "para el 80 %: ", train_class_0.shape[0], "para el 20 %: ", test_class_0.shape[0])
print("para la clase 1 se tiene la cantidad total de: ",data_class_1.shape[0], "para el 80 %: ", train_class_1.shape[0], "para el 20 %: ", test_class_1.shape[0])
print("para la clase 2 se tiene la cantidad total de: ",data_class_2.shape[0], "para el 80 %: ", train_class_2.shape[0], "para el 20 %: ", test_class_2.shape[0])
print("la cantidad total de datos es: ", data.shape[0])

para la clase 0 se tiene la cantidad total de:  59445 para el 80 %:  47556 para el 20 %:  11889
para la clase 1 se tiene la cantidad total de:  18961 para el 80 %:  15168 para el 20 %:  3793
para la clase 2 se tiene la cantidad total de:  21594 para el 80 %:  17275 para el 20 %:  4319
la cantidad total de datos es:  100000


asignamos los datos a X y Y (entrenamiento y prueba) de todas las clases(0,1,2)

In [7]:
X_train_class_0 = train_class_0.drop("class", axis=1)
y_train_class_0 = train_class_0["class"]

X_train_class_1 = train_class_1.drop("class", axis=1)
y_train_class_1 = train_class_1["class"]

X_train_class_2 = train_class_2.drop("class", axis=1)
y_train_class_2 = train_class_2["class"]

X_test_class_0 = test_class_0.drop("class", axis=1)
y_test_class_0 = test_class_0["class"]

X_test_class_1 = test_class_1.drop("class", axis=1)
y_test_class_1 = test_class_1["class"]

X_test_class_2 = test_class_2.drop("class", axis=1)
y_test_class_2 = test_class_2["class"]

Concatenamos los datos separados de todas las clases a una sola class_0   class_1   class_2    --> class
Asi tenemos los datos de entrenamiento en X_train y y_train, y los datos de prueba X_test y y_test

In [8]:
X_train = pd.concat([X_train_class_0, X_train_class_1, X_train_class_2]).values
y_train = pd.concat([y_train_class_0, y_train_class_1, y_train_class_2]).values

indices_train = np.random.permutation(len(X_train))
X_train = X_train[indices_train]
y_train = y_train[indices_train]
m_train = X_train.shape[0]


X_test = pd.concat([X_test_class_0, X_test_class_1, X_test_class_2]).values
y_test = pd.concat([y_test_class_0, y_test_class_1, y_test_class_2]).values

indices_test = np.random.permutation(len(X_test))
X_test = X_test[indices_test]
y_test = y_test[indices_test]
m_test = X_test.shape[0]

#print(X_train)
#print(y_train)

In [9]:
#encontrar el numero de ejemplos de entrenamiento y el numero de caracteristicas
print("la cantidad de ejemplos para entrenamiento es: ", len(X_train))
print("la cantidad de ejemplos para pruebas es: ", len(X_test))
print("la cantidad de caracteristicas es: ", X_train.shape[1])
print("la cantidad de etiquetas es: ", num_labels)
print("la cantidad total de ejemplos: ", data.shape[0])

la cantidad de ejemplos para entrenamiento es:  79999
la cantidad de ejemplos para pruebas es:  20001
la cantidad de caracteristicas es:  15
la cantidad de etiquetas es:  3
la cantidad total de ejemplos:  100000


In [10]:
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

Normalizamos los datos de X, y guardamos los valores de mu y sigma

In [11]:
# llama featureNormalize con los datos cargados
X_norm, mu, sigma = featureNormalize(X_train)

In [12]:
print(X_norm[0,:])
print(y_train)

[ 0.81519733  0.89439067 -0.65768905 -0.83562106 -1.13701464 -1.16370248
 -1.1756315  -0.24226056  0.30597142 -0.06116845  0.31251861 -0.50954046
  0.31248286  0.28101367  1.66268299]
[0 0 0 ... 2 0 0]


In [13]:
m, n = X_norm.shape

# Agraga el termino de intercepción a A
#X = np.concatenate([np.ones((m, 1)), X_norm], axis=1)
X = pd.DataFrame(X_norm)
display(X)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.815197,0.894391,-0.657689,-0.835621,-1.137015,-1.163702,-1.175632,-0.242261,0.305971,-0.061168,0.312519,-0.509540,0.312483,0.281014,1.662683
1,0.237391,0.694168,0.163583,0.892486,0.245288,0.034909,-0.030633,0.011715,1.567303,-1.047920,-0.391104,-0.092270,-0.391109,-0.141870,0.117990
2,-1.602054,-1.560044,-0.719377,-1.141327,-1.392416,-1.418887,-1.441343,1.836370,0.936637,1.791508,0.684106,-0.546502,0.684077,0.572334,1.464551
3,-1.563586,-0.602025,-0.448609,0.125721,0.566142,0.819280,0.819362,1.672482,0.936637,2.006311,2.001248,1.734124,2.001238,1.573988,0.906847
4,-1.610134,-1.243649,-0.522188,-0.272018,-0.056760,0.073786,0.237329,-0.114000,-0.324695,0.522827,-1.374262,-0.541558,-1.374242,-1.641035,-1.235910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79994,0.451307,1.454480,0.976866,0.720280,0.392853,0.148109,0.029044,-0.398005,0.936637,-0.343097,0.534495,-0.011066,0.534492,0.442428,0.301445
79995,-1.596869,-1.303094,-0.394339,0.250212,0.623156,0.760089,1.004292,1.676045,1.567303,-0.853255,0.915931,1.262820,0.915900,0.770785,1.611316
79996,-0.321479,-0.230967,0.341367,-0.543179,-0.987701,-1.280167,-1.433818,0.331348,-0.955361,-0.953944,-0.940051,-0.787904,-0.940039,-1.011961,-0.769934
79997,-1.612286,-1.560054,1.633467,0.614433,0.302144,0.130349,0.143325,1.836370,0.936637,1.744520,0.684086,-0.083670,0.684077,0.572334,0.587635


In [14]:
def sigmoid(z):
    """
    Calcula la sigmoide de z.
    """
    z = np.clip(z, -500, 500)
    return 1.0 / (1.0 + np.exp(-z))

In [15]:
def lrCostFunction(theta, X, y, lambda_):
#     alpha = 0.003
#     theta = theta.copy()
    # Inicializa algunos valores utiles
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0

#     J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))
    #J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
    J = (1 / m) * np.sum(-y.dot(np.log(h + 1e-10)) - (1 - y).dot(np.log(1 - h + 1e-10))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
#     theta = theta - (alpha / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad
#    return J, theta

In [16]:
def oneVsAll(X, y, num_labels, lambda_):

    # algunas variables utiles
    m, n = X.shape

    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta

Entrenamos el modelo y calculamos las thetas con la funcion creada oneVsAll

In [17]:
lambda_ = 0.1
all_theta = oneVsAll(X_norm, y_train, num_labels, lambda_)

In [18]:
for i in range(all_theta.shape[1]):
    print("Theta para la clase 0", i, " es: ", all_theta[0][i], "Theta para la clase 1", i, " es: ", all_theta[1][i], "Theta para la clase 2", i, " es: ", all_theta[2][i])

Theta para la clase 0 0  es:  0.5777016471431513 Theta para la clase 1 0  es:  -2.554960340159344 Theta para la clase 2 0  es:  -19.606894210278462
Theta para la clase 0 1  es:  0.031129789005994016 Theta para la clase 1 1  es:  0.10778695664333549 Theta para la clase 2 1  es:  -0.11237089495524451
Theta para la clase 0 2  es:  -0.051928302637649866 Theta para la clase 1 2  es:  0.28041851601026596 Theta para la clase 2 2  es:  -0.0954053397652406
Theta para la clase 0 3  es:  -0.07078225695836901 Theta para la clase 1 3  es:  -1.209661268202378 Theta para la clase 2 3  es:  0.6358363112525913
Theta para la clase 0 4  es:  3.7230040524117154 Theta para la clase 1 4  es:  -1.6643177399554578 Theta para la clase 2 4  es:  -0.2576039619837537
Theta para la clase 0 5  es:  -1.285997169202123 Theta para la clase 1 5  es:  -0.5325683254766356 Theta para la clase 2 5  es:  -0.013231226871329629
Theta para la clase 0 6  es:  -1.7405339910244997 Theta para la clase 1 6  es:  1.6516888652585946 

In [19]:
print(all_theta)

[[ 5.77701647e-01  3.11297890e-02 -5.19283026e-02 -7.07822570e-02
   3.72300405e+00 -1.28599717e+00 -1.74053399e+00 -2.81890597e-01
   8.45598108e-02 -7.43024154e-02  7.25248914e-02  5.40057769e-01
  -2.78102695e-01  5.40059848e-01 -1.21334806e+00  3.82267667e-02]
 [-2.55496034e+00  1.07786957e-01  2.80418516e-01 -1.20966127e+00
  -1.66431774e+00 -5.32568325e-01  1.65168887e+00  5.77930098e-01
   2.95274082e-03  3.00043577e-02 -3.32719692e-03 -4.44314252e-01
   4.69695460e+00 -4.44317724e-01  5.84390567e-01  4.69448015e-02]
 [-1.96068942e+01 -1.12370895e-01 -9.54053398e-02  6.35836311e-01
  -2.57603962e-01 -1.32312269e-02 -1.92314566e-01  4.83555226e-01
  -1.69894428e-01  7.92310915e-02 -5.43361113e-02 -8.23788413e-01
  -2.80081747e+01 -8.23791300e-01  1.97950793e+00 -7.10111569e-02]]


<a id="section3"></a>
#### 1.4.1 Prediccion One-vs-all


In [20]:
def predictOneVsAll(all_theta, X):

    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

Se prueba el modelo con los datos x_test y se calcula la precision del entrenamiento con ambos datos, es decir con los de entrenamiento y los de prueba

In [21]:
#Realiamos la prediccion y evaluamos la precision para un modelo de regresion logistica
print(X.shape)

pred = predictOneVsAll(all_theta, X_norm)
print('Precision del conjuto de entrenamiento haciendo uso del 80% de datos: {:.2f}%'.format(np.mean(pred == y_train) * 100))

X_test_norm = (X_test - mu) / sigma
p = predictOneVsAll(all_theta, X_test_norm)
print('Precision del conjuto de entrenamiento haciendo uso de 20% de datos: {:.2f}%'.format(np.mean(p == y_test) * 100))


for i in range(30):
    print("Prediccion: ", p[i], "Valor real: ", y_test[i])

(79999, 15)
Precision del conjuto de entrenamiento haciendo uso del 80% de datos: 92.73%
Precision del conjuto de entrenamiento haciendo uso de 20% de datos: 93.00%
Prediccion:  0 Valor real:  0
Prediccion:  0 Valor real:  0
Prediccion:  0 Valor real:  0
Prediccion:  0 Valor real:  0
Prediccion:  0 Valor real:  0
Prediccion:  0 Valor real:  0
Prediccion:  2 Valor real:  2
Prediccion:  1 Valor real:  1
Prediccion:  1 Valor real:  0
Prediccion:  2 Valor real:  2
Prediccion:  1 Valor real:  1
Prediccion:  0 Valor real:  1
Prediccion:  2 Valor real:  2
Prediccion:  1 Valor real:  1
Prediccion:  0 Valor real:  0
Prediccion:  2 Valor real:  2
Prediccion:  1 Valor real:  1
Prediccion:  0 Valor real:  0
Prediccion:  0 Valor real:  0
Prediccion:  0 Valor real:  0
Prediccion:  0 Valor real:  0
Prediccion:  0 Valor real:  0
Prediccion:  0 Valor real:  0
Prediccion:  0 Valor real:  0
Prediccion:  2 Valor real:  2
Prediccion:  2 Valor real:  2
Prediccion:  2 Valor real:  2
Prediccion:  0 Valor real